In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import linear_model
from sklearn.isotonic import IsotonicRegression
from ideas import intersection_over_union
from ideas import bb_intersection_over_union
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import cross_val_score
from ideas import al
from ideas import aw
import math
from sklearn.ensemble import BaggingRegressor
from itertools import combinations
from mlens.ensemble import SuperLearner

[MLENS] backend: threading


In [39]:
columns = ['Xmin','Ymin','Xmax','Ymax']
box_columns = columns
box_columns_gd = list(map(lambda x: x + '_gd', box_columns))

center_columns = ['X', 'Y', 'XYD', 'area', 'XYDMax', 'A', 'P']
# 'L', 'W', 'Xcenter', 'Ycenter'
l, w, xc, yc = 0,1,2,3 
center_columns_gd = list(map(lambda x: x + '_gd', center_columns))

feature_columns = list(map(lambda x: 'delta' + x, columns + ['min', 'max']))
feature_ratio_columns = []

columns_all = box_columns + box_columns_gd + center_columns + feature_columns
predict_columns = columns_all + feature_ratio_columns
columns_true = list(map(lambda x: x + '_true', columns))
columns_id = columns + ['itemId']
theta = 0

In [3]:
def weighted_averaging(iid, df, np_data, avg, columns, back_up = None):
    global box_columns
    items = df[df['itemId'] == iid]
    vi = np_data[items.index.values]
    bl = len(columns)
    avg.loc[iid][bl] = iid
    if vi.shape[0] == 0:
        bp = back_up[back_up['itemId'] == iid][box_columns]
        for y in range(0, bl):                
            avg.loc[iid][y] = bp[box_columns[y]]               
        return
    l = al(vi)
    w = aw(vi)
    sabx = (l * w) ** 2
    sb = sum(1 / sabx)    
    if math.isinf(sb):
        vi = vi.mean(axis=0)
        for y in range(0, bl):                
            avg.loc[iid][y] = vi[y]
    else:
        for y in range(0, bl):                
            avg.loc[iid][y] = sum(vi[:,y] / sabx) / sb
    
    return avg.loc[iid]

def zip_stacked_classifiers(*args):
    to_zip = []
    for arg in args:
        combined_items = sum([map(list, combinations(arg, i)) for i in range(len(arg) + 1)], [])
        combined_items = filter(lambda x: len(x) > 0, combined_items)
        to_zip.append(combined_items)
    
    return zip(to_zip[0], to_zip[1])

def custom_cv_2folds(X):
    v, theta = 0, .2    
    while(v <= 1):
        idx = X[(X['XYD'] >= v) & (X['XYD'] < v + theta)].index
        yield idx, idx
        v = v + theta
        
def my_custom_loss_func(y_true, y_pred):
    iou = np.zeros(len(y_true))
    for i in range(0, len(y_pred)):
        if type(y_true) is np.ndarray:
            iou[i] = bb_intersection_over_union(y_true, y_pred[i])                          
        else:
            iou[i] = bb_intersection_over_union(y_true.iloc[i].values, y_pred[i]) 
    res = iou.mean()
    return res

In [48]:
te_d = pd.read_csv('test_data_full.csv')
t_d = pd.read_csv('train_data_full.csv')
t_a = pd.read_csv('train_answers.csv')
removeIds = []
t_d_a = t_d.merge(t_a, on="itemId")
t_d_a = t_d_a.drop(t_d_a[t_d_a['itemId'].isin(removeIds)].index)
ids = list(set(te_d['itemId']))
scaler = MinMaxScaler()
scaler_fit_d = t_d_a[columns_all].values
scaler_fit_d = np.append(scaler_fit_d, te_d[columns_all].values, axis = 0)
scaler.fit(scaler_fit_d)
t_d_a[columns_all] = scaler.transform(t_d_a[columns_all])
t_d_a = t_d_a.sort_values(['XYD'])
te_d[columns_all] = scaler.transform(te_d[columns_all])
t_d_a = t_d_a[t_d_a['score'] > theta]

In [49]:
seed = 49
np.random.seed(seed)
X = t_d_a[predict_columns]
y = t_d_a
kf = KFold(n_splits=5, random_state = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,\
                                                    random_state = seed)
custom_cv = custom_cv_2folds(X_train.reset_index())
clf = linear_model.RidgeCV(alphas=np.logspace(-4, 0, 5), cv=kf,\
                           scoring=make_scorer(my_custom_loss_func))
# clf = linear_model.MultiTaskElasticNetCV(l1_ratio=0.95, eps=0.0001, cv=kf,\
# #                                          alphas=np.logspace(-4, 0, 2),\
#                                          max_iter=10000, random_state = seed)
bagging_clf = BaggingRegressor(clf, max_samples=0.3, random_state=seed, n_jobs=-1)
bagging_clf.fit(X_train, y_train[columns_true])
y_test_pred = bagging_clf.predict(X_test)

iou = np.zeros(len(y_test_pred))
y_test_pred_pd = pd.DataFrame(data=np.c_[y_test_pred, y_test['itemId']], columns=columns_id)
y_test_pred_pd['itemId'] = pd.to_numeric(y_test_pred_pd['itemId'], downcast='integer')
data = y_test[columns_true + ['itemId']].merge(y_test_pred_pd, on=['itemId'])
data["iou"] = data[columns + columns_true].apply(intersection_over_union, axis=1)
print(data["iou"].mean())

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [42]:
ids = list(set(data['itemId']))
print(set(data[data['iou'] < .1]['itemId']))
# print(data[data['itemId']  == 25135])

{32928, 26628, 22483, 25142, 31033, 35738}


In [43]:
seed = 79
np.random.seed(0)
kf = KFold(n_splits=10, random_state = seed)
jobs = 1
lasso = linear_model.MultiTaskLassoCV(alphas=np.logspace(-4, 0, 5), max_iter=10000, n_jobs=jobs,\
                                      random_state = seed, cv=kf)
ridge = linear_model.RidgeCV(alphas=np.logspace(-4, 0, 5), cv=kf)
net = linear_model.MultiTaskElasticNetCV(l1_ratio=0.9, eps=0.0001,\
                                        max_iter=10000, random_state = seed, cv=kf)
clf_array = [lasso, ridge, net]
best_combination = [0.00, ""]
X = t_d_a[predict_columns]
y = t_d_a
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,\
                                                    random_state = seed)
clf_array = [lasso, ridge, net]
names = ['MultiTaskLassoCV', 'RidgeCV', 'MultiTaskElasticNetCV']
stacked_clf_list = list()
for i in range(1, len(names) + 1):
    nl = list(map(list, combinations(names, i)))
    clfl = list(map(list, combinations(clf_array, i)))
    stacked_clf_list = stacked_clf_list + list(zip(clfl, nl))
    
for clf in stacked_clf_list:    
    ensemble = SuperLearner(scorer = make_scorer(my_custom_loss_func), 
                            random_state = seed, 
                            folds = 10)
    ensemble.add(clf[0])
    ensemble.fit(X_train, y_train)
    preds = ensemble.predict(X_test)
    accuracy = my_custom_loss_func(preds, y_test)
    
    if accuracy > best_combination[0]:
        best_combination[0] = accuracy
        best_combination[1] = clf[1]
    
    print("Accuracy score: {:.3f} {}").format(accuracy, clf[1])

    print("\nBest stacking model is {} with accuracy of: {:.3f}").\
    format(best_combination[1], best_combination[0])

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.6/runpy.py in _run_code(code=<

In [ ]:
seed = 49
np.random.seed(seed)
kf = KFold(n_splits=10, random_state = seed, shuffle=True)
lasso = linear_model.MultiTaskLassoCV(alphas=np.logspace(-4, 0, 5), cv=kf,\
                                      max_iter=10000, n_jobs=-1, random_state = seed)
ridge = linear_model.RidgeCV(alphas=np.logspace(-4, 0, 5), cv=kf)
net = linear_model.MultiTaskElasticNetCV(l1_ratio=0.9, cv=kf, eps=0.0001,\
                                        max_iter=10000, random_state = seed)

clf_array = [lasso, ridge, net]
X = t_d_a[predict_columns]
y = t_d_a
for clf in clf_array:
    vanilla_scores = cross_val_score(clf, X, y[columns_true], cv=kf, n_jobs=-1,\
                                     scoring=make_scorer(my_custom_loss_func))
    bagging_clf = BaggingRegressor(clf, max_samples=0.4, random_state=seed, n_jobs=-1)
    bagging_scores = cross_val_score(bagging_clf, X, y[columns_true], cv=kf, n_jobs=-1,\
                                     scoring=make_scorer(my_custom_loss_func))    
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]\n".\
          format(clf.__class__.__name__, vanilla_scores.mean(), vanilla_scores.std()))
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".\
          format(clf.__class__.__name__, bagging_scores.mean(), bagging_scores.std()))

In [45]:
X_submit = te_d[predict_columns]
y_submit = bagging_clf.predict(X_submit)
y_submit_pd = pd.DataFrame(data=np.c_[y_submit, te_d['itemId']], columns=columns_id)
y_submit_pd['itemId'] = pd.to_numeric(y_submit_pd['itemId'], downcast='integer')
result = y_submit_pd
# ids = list(set(result['itemId']))
# answers = pd.DataFrame(np.zeros((len(ids), len(box_columns_id)), dtype=int), index=ids, columns=box_columns_id)
# y_submit = result.values
# for iid in ids:
#     weighted_averaging(iid, result, y_submit, answers, columns)
answers = result.groupby('itemId').mean().reset_index()
answers[['itemId'] + columns].to_csv('submit2704.csv', index=False, header=False)
data = answers.merge(pd.read_csv("submit-max.csv"), on=['itemId'])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
data["iou"].mean()

0.8406873158164314